In [ ]:
import tempfile
from pathlib import Path
from subprocess import PIPE, call

import aiger
#from aiger import utils


SIMPLIFY_TEMPLATE = 'read {0}; dc2; dc2; dc2; fraig; write_aiger -s {0}'
SIMPLIFY_TEMPLATE_2 = '&r {0}; &put; fold; pdr -v'


def simplify(circ, verbose=False, abc_cmd='yosys/yosys-abc', aigtoaig_cmd='aiger_tool_util/aigtoaig'):
    circ = aiger.to_aig(circ)

    # avoids confusion and guarantees deletion on exit
    with tempfile.TemporaryDirectory() as tmpdirname:
        tmpdir = Path(tmpdirname)
        aag_path = tmpdir / 'input.aag'
        aig_path = tmpdir / 'input.aig'

        circ.write(aag_path)
        call([aigtoaig_cmd, aag_path, aig_path])
        command = [
            abc_cmd,
            '-c',
            SIMPLIFY_TEMPLATE.format(aig_path)
        ]
        call(command) if verbose else call(command, stdout=PIPE)
        call([aigtoaig_cmd, aig_path, aag_path])
        return aiger.parser.load(aag_path)

def load_aig(aig_path,aigtoaig_cmd='aiger_tool_util/aigtoaig'):
    with tempfile.TemporaryDirectory() as tmpdirname:
        tmpdir = Path(tmpdirname)
        aag_path = tmpdir / 'input.aag'
        call([aigtoaig_cmd, aig_path, aag_path])
        return aiger.parser.load(aag_path)

def aig_mc(circ,abc_cmd='yosys/yosys-abc', aigtoaig_cmd='aiger_tool_util/aigtoaig'):
    with tempfile.TemporaryDirectory() as tmpdirname:
        tmpdir = Path(tmpdirname)
        aag_path = tmpdir / 'input.aag'
        aig_path = tmpdir / 'input.aig'
        circ.write(aag_path)
        call([aigtoaig_cmd, aag_path, aig_path])
        command = [
            abc_cmd,
            '-c',
            SIMPLIFY_TEMPLATE_2.format(aig_path)
        ]
        call(command)

def sat_solving(cnf_path,minisat_cmd='minisat/build/release/bin/minisat'):
    command = [
        minisat_cmd,
        cnf_path
    ]
    call(command)

f = load_aig('benchmarks/arithmetic/hyp.aig')
#f = load_aig('benchmarks/random_control/voter.aig')
#f2 = simplify(f)
#print(f.aig)
aig_mc(f)
#aig_mc(f2)

# x = aiger.atom('x')
# f = x ^ x
# print(f.aig)

# f2 = simplify(f)
# print(f2.aig)
